# Online Correlated Selection Implementation(OCS)

The following is the implementation of the Online Primal-Dual Edge-Weighted Bipartite Matching Algorithm (Algorithm 1)
with Online Correlated Selection (OCS) (Algorithm 2), as described in the paper "Edge-Weighted Online Bipartite Matching" by Matthew Fahrbach, Zhiyi Huang, Runzhou Tao, and Morteza Zadimoghaddam (2020).

The algorithm is implemented in Python and the competitive ratio is calculated by comparing the algorithm's objective value to the optimal offline matching objective value. The algorithm is tested on random examples and the competitive ratio is calculated. The algorithm is also compared to the greedy algorithm and the competitive ratio is calculated.

Original Paper: https://arxiv.org/abs/2002.08536


In [1]:
import numpy as np
from pulp import *
from scipy.optimize import linear_sum_assignment

## Optimizing the Gain-Sharing Parameters (4.4 in the paper)

The gain-sharing parameters $a(k)$ and $b(k)$ are optimized by solving a linear program. Note that these gain-sharing values $a(k)$ and $b(k)$ are instance independent (i.e., they do not depend on the underlying graph and are used globally) and defined for all $k ∈ Z≥0$.

We directly use the same parameters as in the paper, i.e., $\gamma = 1/16$, $k_{max} = 8$, and $\kappa = 1.5$. We set $a(k) = b(k) = 0$ for all $k > kmax$ for some sufficiently large integer $kmax$.

Official Implementation (of this section only): https://github.com/fahrbach/edge-weighted-online-bipartite-matching

In [2]:
gamma = 1/16
kmax = 8
kappa = 1.5

def dx_ocs(gamma, kmax):
    """
    The increment of the probability of selecting the node at least once in k rounds compared to k-1 rounds
    :param gamma: a discount factor
    :param kmax: the maximum number of rounds
    :return:
    """
    # x[k] = 1 - 0.5^k * (1-gamma)^(max(0, k-1)) # k = 1, 2, ..., kmax+1
    # x[k]: the probability of selecting the first node in the online correlated selection
    x = np.array([1 - pow(0.5, k)*pow(1-gamma, max(0,k-1)) for k in range(kmax+2)])
    return x[1:] - x[:kmax+1]

def ratio_lp(gamma, kmax, kappa=1.5):
    #dx = dx_ocs(gamma, kmax)
    #print('dx:', dx)

    lp = LpProblem('ratio lp', LpMaximize)

    Gamma = LpVariable('Γ', 0.)
    lp += Gamma

    a = LpVariable.dict('a', range(kmax+1), 0.)
    b = LpVariable.dict('b', range(kmax+1), 0.)

    # gain split in deterministic rounds
    for k in range(kmax+1):
        #lp += sum([a[i] for i in range(k, kmax+1)]) + kappa*b[k] <= sum(dx[k:])
        lp += sum([a[i] for i in range(k, kmax+1)]) + kappa*b[k] <= pow(0.5, k)*pow(1-gamma, max(k-1, 0))

    # gain split in randomized rounds
    lp += a[0] + b[0] <= 0.5
    for k in range(1, kmax+1):
        #lp += a[k] + b[k] <= dx[k]
        lp += a[k] + b[k] <= pow(0.5, k+1)*pow(1-gamma, k-1)*(1+gamma)

    # compensation
    lp += a[0] >= gamma/2

    # boundary condition
    lp += sum([a[i] for i in range(kmax+1)]) >= Gamma

    # approximate dual feasibility: randomized round, not chosen
    for k in range(kmax+1):
        lp += sum([a[i] for i in range(k)]) + 2*b[k] >= Gamma

    # approximate dual feasibility: randomized round, chosen
    for k in range(kmax):
        lp += sum([a[i] for i in range(k+1)]) + kappa*b[k] >= Gamma

    lp.solve()

    return (value(Gamma),
            [value(a[k]) for k in range(kmax+1)],
            [value(b[k]) for k in range(kmax+1)])

Gamma, a, b = ratio_lp(gamma, kmax, kappa)
print(Gamma, a, b)

# add 0 to the end of a and b until 1000
a += [0] * (2000 - len(a))
b += [0] * (2000 - len(b))

/Users/mai/anaconda3/envs/optim/lib/python3.10/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mai/anaconda3/envs/optim/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/v0/x6q_frpj36x_0ddd6y0f1pw80000gn/T/c50d12711100443a8521968506cdccae-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/v0/x6q_frpj36x_0ddd6y0f1pw80000gn/T/c50d12711100443a8521968506cdccae-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 42 COLUMNS
At line 233 RHS
At line 271 BOUNDS
At line 272 ENDATA
Problem MODEL has 37 rows, 19 columns and 189 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 36 (-1) rows, 19 (0) columns and 188 (-1) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 0.00088335093 ( 0.088335093%) - largest zero change 0.00085623016
0  Obj -2.6757193e-05 Dual inf 0.99911565 (1)
23  Obj 0.50413042
Optimal - objective value 0.50503489
After Posts

## Offline Matching and Greedy Algorithm

In [3]:
# optimal found by offline matching
def offline_matching(left_nodes, right_nodes, edges):
    left_nodes = list(left_nodes)
    right_nodes = list(right_nodes)
    cost_matrix = np.zeros((len(left_nodes), len(right_nodes)))

    for i, left_node in enumerate(left_nodes):
        for j, right_node in enumerate(right_nodes):
            for edge, weight in edges.items():
                if edge[0] == left_node and edge[1] == right_node:
                    cost_matrix[i, j] = -weight
                    break

    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    matching = {}
    for i, j in zip(row_ind, col_ind):
        matching[left_nodes[i]] = (right_nodes[j], -cost_matrix[i, j])
    objective = -cost_matrix[row_ind, col_ind].sum()

    return matching, objective

# greedy algorithm
def deterministic_greedy(left_nodes, right_nodes, edges):
    matching = {}
    weights = {left_node: 0 for left_node in left_nodes}

    for right_node in right_nodes:
        gains = {}
        # select weight that maximizes the gain = weight - previous weight
        for left_node in left_nodes:
            for edge, weight in edges.items():
                if edge[0] == left_node and edge[1] == right_node:
                    gains[left_node] = weight - weights[left_node]
        # find the left node that maximizes the gain
        if gains == {}:
            continue

        max_left_node = max(gains, key=gains.get)
        max_gain = gains[max_left_node]
        if max_gain > 0:
            weights[max_left_node] += max_gain
            matching[max_left_node] = (right_node, weights[max_left_node])

    objective = sum([weight for left_node, (right_node, weight) in matching.items()])

    return matching, objective

## example
## optimal matching should be (1-2), (2-1), (3-3)
left_nodes = {1, 2, 3}
right_nodes = {1, 2, 3}
edges = {(1, 1): 10, (2, 1): 30, (3, 1): 0,
         (1, 2): 20, (2, 2): 0, (3, 2): 50,
         (1, 3): 0, (2, 3): 40, (3, 3): 60}

matching, alg = deterministic_greedy(left_nodes, right_nodes, edges)
print("Greedy Matching:")
for left_node, (right_node, weight) in matching.items():
    print(f"Left: {left_node}, Right: {right_node}, Weight: {weight}")
print("Greedy Objective:", alg)

matching, obj = offline_matching(left_nodes, right_nodes, edges)
print("Optimal Matching:")
for left_node, (right_node, weight) in matching.items():
    print(f"Left: {left_node}, Right: {right_node}, Weight: {weight}")
print("Optimal Objective:", obj)

print("Competitive Ratio:", alg / obj)

Greedy Matching:
Left: 2, Right: 3, Weight: 40
Left: 3, Right: 2, Weight: 50
Greedy Objective: 90
Optimal Matching:
Left: 1, Right: 2, Weight: 20.0
Left: 2, Right: 1, Weight: 30.0
Left: 3, Right: 3, Weight: 60.0
Optimal Objective: 110.0
Competitive Ratio: 0.8181818181818182


In [38]:
## Simulate random example for greedy algorithm.
# np.random.seed(100)
left_nodes = set([i for i in range(1, 100)])
right_nodes = set([i for i in range(1, 100)])
edges = {(np.random.choice(list(left_nodes)), np.random.choice(list(right_nodes))):
             np.random.randint(0, 2) for _ in range(3000)}

# print("edges", edges)

## Add 0 weighted-edges
for left_node in left_nodes:
    for right_node in right_nodes:
        if (left_node, right_node) not in edges:
            edges[(left_node, right_node)] = 0

matching, alg = deterministic_greedy(left_nodes, right_nodes, edges)
print("Greedy Matching:")
for left_node, (right_node, weight) in matching.items():
    print(f"Left: {left_node}, Right: {right_node}, Weight: {weight}")
print("matching", matching)
print("Greedy Objective:", alg)

matching, obj = offline_matching(left_nodes, right_nodes, edges)
print("Optimal Matching:")
for left_node, (right_node, weight) in matching.items():
    print(f"Left: {left_node}, Right: {right_node}, Weight: {weight}")
print("Optimal Objective:", obj)

print("Competitive Ratio:", alg / obj)

Greedy Matching:
Left: 3, Right: 1, Weight: 1
Left: 11, Right: 2, Weight: 1
Left: 4, Right: 3, Weight: 1
Left: 2, Right: 4, Weight: 1
Left: 6, Right: 5, Weight: 1
Left: 7, Right: 6, Weight: 1
Left: 1, Right: 7, Weight: 1
Left: 8, Right: 8, Weight: 1
Left: 29, Right: 9, Weight: 1
Left: 13, Right: 10, Weight: 1
Left: 27, Right: 11, Weight: 1
Left: 22, Right: 12, Weight: 1
Left: 12, Right: 13, Weight: 1
Left: 16, Right: 14, Weight: 1
Left: 19, Right: 15, Weight: 1
Left: 15, Right: 16, Weight: 1
Left: 44, Right: 17, Weight: 1
Left: 34, Right: 18, Weight: 1
Left: 24, Right: 19, Weight: 1
Left: 25, Right: 20, Weight: 1
Left: 42, Right: 21, Weight: 1
Left: 32, Right: 22, Weight: 1
Left: 17, Right: 23, Weight: 1
Left: 31, Right: 24, Weight: 1
Left: 21, Right: 25, Weight: 1
Left: 30, Right: 26, Weight: 1
Left: 18, Right: 27, Weight: 1
Left: 20, Right: 28, Weight: 1
Left: 5, Right: 29, Weight: 1
Left: 46, Right: 30, Weight: 1
Left: 23, Right: 31, Weight: 1
Left: 9, Right: 32, Weight: 1
Left: 41,

## Primal-Dual Algorithm with Online Correlated Selection (OCS)

Implementation of Algorithm 1 (section 4.1) and Algorithm 2 (section 4.4) in the paper.

In [48]:
### Algorithm 2: Online Correlated Selection (OCS)
verbose = False
# On receive 2 candidate nodes:
def online_correlated_selection(node_1, node_2, tau_i):
    # (1) With probability 1/2:
    if np.random.rand() < 0.5:
        # (a) Draw l, m ∈ {1, 2} uniformly at random.
        l = np.random.choice([node_1, node_2])
        m = np.random.choice([node_1, node_2])
        # (b) Let tau_[-m] = "unknown"
        if m == node_1:
            tau_i[node_2] = "unknown"
        else:
            tau_i[node_1] = "unknown"
        # (c) If m = l, let tau_m = "selected";
        # otherwise, let tau_m = "not selected".
        if m == l:
                tau_i[m] = "selected"
        else:
                tau_i[m] = "not selected"

    # (2) Otherwise, (i.e. with probability 1/2):
    else:
        # (a) Draw m ∈ {1, 2} uniformly at random.
        m = np.random.choice([node_1, node_2])
        # (b) If tau_m = "selected", let l = -m;
        if tau_i[m] == "selected":
            l = node_1 if m == node_2 else node_2
        # If tau_m = "not selected", let l = m;
        elif tau_i[m] == "not selected":
            l = m
        # If tau_m = "unknown", draw l ∈ {1, 2} uniformly at random.
        else:
            l = np.random.choice([node_1, node_2])
        # (c) Let tau_node_1 = tau_node_2 = "unknown".
        tau_i[node_1] = "unknown"
        tau_i[node_2] = "unknown"

    # (3) Select l.
    return l, tau_i

### Algorithm 1: Online Primal-Dual Edge-Weighted Bipartite Matching Algorithm
# Compute the gain split in randomized rounds and deterministic rounds
def randomized_gain_split(left_node, right_node, k, edges):
    w_ij = edges[(left_node, right_node)]
    max_weight = max(edges.values())

    # # int_{0}^{w_ij} b(ki(w))dw
    # part_1 = sum([b[k[left_node][w]] for w in k[left_node] if 0 < w <= weight_ij])
    # # int_{w_ij}^{inf} [sum_{0<=l<ki(w)}a(l)] dw
    # part_2 = sum([a[l] for w in k[left_node] if w >= weight_ij > 0
    #                     for l in range(k[left_node][w])])
    part_1 = 0
    for w in range(0, w_ij+1):
        part_1 += b[k[left_node][w]]
        # print(f"b[k]{w}", k[left_node][w], b[k[left_node][w]])
    # k_wij = k[left_node][w_ij]
    # part_1 = sum(b[k] for k in range(0, k_wij + 1))
    part_2 = 0
    for w in range(w_ij + 1, max_weight+1):  # You can adjust the upper limit as needed
        part_2 += sum(a[l] for l in range(k[left_node][w]))
    if verbose: print("part_1", part_1, "part_2", part_2/2)
    return part_1 - 0.5 * part_2

def deterministic_gain_split(left_node, right_node, k, edges):
    return kappa * randomized_gain_split(left_node, right_node, k, edges)

def ocs_matching(left_nodes, right_nodes, edges):

    ## Initialize
    matching = {}
    # k = {left_node: {weight: 0 for weight in
    #                  set([weight for edge, weight in edges.items() if edge[0] == left_node])}
    #                  for left_node in left_nodes}
    max_weight = max(edges.values())
    k = {left_node: {weight: 0 for weight in range(max_weight+1)} for left_node in left_nodes}

    tau_i = {left_node: 0 for left_node in left_nodes} # 0: unknown, 1: selected, 2: not selected
    # print("edges", edges)
    ## Matching: For each incoming right node
    for right_node in right_nodes:
        # # skip if all adjacent left nodes are 0.
        if all([edges[(left_node, right_node)] == 0 for left_node in left_nodes]):
            continue
        # (1) For every offline vertex i ∈ L,
        # compute ΔiR βj and ΔiD βj according to Equations (5) and (6).
        # delta_R = {left_node: 0 for left_node in left_nodes}
        # delta_D = {left_node: 0 for left_node in left_nodes}

        # delta_R only the ones that are adjacent to right node
        delta_R = {left_node: 0 for left_node in left_nodes if edges[(left_node, right_node)] != 0}
        delta_D = {left_node: 0 for left_node in left_nodes if edges[(left_node, right_node)] != 0}

        # find adjacent left nodes

        for left_node in left_nodes:
            for edge, weight in edges.items():
                if edge[0] == left_node and edge[1] == right_node and weight != 0:
                    delta_R[left_node] = randomized_gain_split(
                        left_node, right_node, k, edges)
                    delta_D[left_node] = deterministic_gain_split(
                        left_node, right_node, k, edges)
        if verbose:
            print("delta_R", delta_R)
            print("delta_D", delta_D)
        # (2) Find i1 and i2 with the maximum ΔiRβj.

        set_i = sorted([left_node for left_node, gain in
                 delta_R.items() if gain == max(delta_R.values())])

        if len(set_i) > 1:
            i1 = set_i[0]
            i2 = set_i[1]
        else:
            i1 = set_i[0]
            i2 = set_i[0]

        # (3) Find i∗ with the maximum ΔiDβj
        i_star = max(delta_D, key=delta_D.get)
        # print("SET I", set_i, i_star)
        Delta_R_i1 = randomized_gain_split(i1, right_node, k, edges)
        Delta_R_i2 = randomized_gain_split(i2, right_node, k, edges)
        Delta_D_i_star = deterministic_gain_split(i_star, right_node, k, edges)

        # (4) If ΔRβ + ΔRβ < 0 and ΔDβ < 0, leave j unmatched.
        if Delta_R_i1 + Delta_R_i2 < 0 and Delta_D_i_star < 0:
            # This corner case occurs when all neighbors of right node
            # are already matched deterministically.
            pass

        # (5) If ΔRβ +ΔRβ ≥0 and ΔRβ +ΔRβ ≥ ΔD, let the OCS pick one of i1 and i2.
        elif Delta_R_i1 + Delta_R_i2 >= 0 \
                and Delta_R_i1 + Delta_R_i2 >= Delta_D_i_star:

            # if edge weight is 0, skip
            # if edges[(i1, right_node)] == 0 and edges[(i2, right_node)] == 0:
            #     print("SKIPPP")
            #     continue

            # Match i1 or i2 to the right node using OCS
            # i_ocs, tau_i = online_correlated_selection(i1, i2, tau_i)
            if verbose: print(tau_i)
            node_1 = i1
            node_2 = i2

            ###
                # (1) With probability 1/2:
            if np.random.rand() < 0.5:

                # (a) Draw l, m ∈ {1, 2} uniformly at random.
                l = np.random.choice([node_1, node_2])
                m = np.random.choice([node_1, node_2])
                # (b) Let tau_[-m] = "unknown"
                if m == node_1:
                    tau_i[node_2] = 0 # "unknown"
                else:
                    tau_i[node_1] = 0 # "unknown"
                # (c) If m = l, let tau_m = "selected";
                # otherwise, let tau_m = "not selected".
                if m == l:
                        tau_i[m] = 1 # "selected"
                else:
                        tau_i[m] = 2 #"not selected"

                if verbose: print("node ", right_node, "oblivious step choose ", l, "from ", node_1, node_2)

            # (2) Otherwise, (i.e. with probability 1/2):
            else:

                # (a) Draw m ∈ {1, 2} uniformly at random.
                m = np.random.choice([node_1, node_2])
                # (b) If tau_m = "selected", let l = -m;
                if tau_i[m] == 1: # "selected":
                    l = node_1 if m == node_2 else node_2
                # If tau_m = "not selected", let l = m;
                elif tau_i[m] == 2: # "not selected":
                    l = m
                # If tau_m = "unknown", draw l ∈ {1, 2} uniformly at random.
                else:
                    l = np.random.choice([node_1, node_2])
                # (c) Let tau_node_1 = tau_node_2 = "unknown".
                tau_i[node_1] = 0 # "unknown"
                tau_i[node_2] = 0 # "unknown"

                if verbose: print("node ", right_node, "adaptive step choose", l, "from ", node_1, node_2)

            i_ocs = l


            ###

            weight_ij = edges[(i_ocs, right_node)]
            matching[i_ocs] = (right_node, weight_ij)

            # Update k_i(w) for i1 and i2
            weight_1 = edges[(i1, right_node)]
            weight_2 = edges[(i2, right_node)]

            if i1 != i2:
                for w in k[i1]:
                    if w <= weight_1 :
                        k[i1][w] += 1

                for w in k[i2]:
                    if w <= weight_2:
                        k[i2][w] += 1

            else:
                for w in k[i1]:
                    if w <= weight_1:
                        k[i1][w] += 1


            if verbose:
                print(f"k_{i1}", k[i1])
                print(f"k_{i2}", k[i2])

        # (6) If ΔDβ ≥0 and ΔDβ > ΔRβ + ΔRβ, match j to i∗.
        # To handle the case when i1 = i2 = i_star, use "else" instead of "elif".
        # elif Delta_D_i_star >= 0 and Delta_D_i_star > Delta_R_i1 + Delta_R_i2:

        else:
            # if edge weight is 0, skip
            if edges[(i_star, right_node)] == 0:
                continue

            weight_ij = edges[(i_star, right_node)]
            matching[i_star] = (right_node, weight_ij)

            # Let ki (w) = ∞ if it has been chosen in a deterministic round
            # in which its edge weight is at least w.
            for w in k[i_star]:
                if w >= weight_ij:
                    k[i_star][w] = 9 # infinity

            if verbose:
                print("node ", right_node, "deterministically choose ", i_star, "at weight", weight_ij)
                print(f"k_{i_star}", k[i_star])

    objective = sum([weight for left_node, (right_node, weight) in matching.items()])
    # if verbose: print("Matching:", matching)

    # print("Matching:", matching)
    return matching, objective

## Hard Instances

### 2x2 Upper Triangular Graph

Greedy achieves a competitive ratio of 0.5, while OCS achieves 0.75 on average.

In [39]:
## Triangular example
left_nodes = [1, 2]
right_nodes = [1, 2]
edges = {(1, 1): 1, (2, 1): 1,
         (1, 2): 1, (2, 2): 0,
         (1, 3): 0, (2, 3): 0}

matching, alg = deterministic_greedy(left_nodes, right_nodes, edges)
print("Greedy Matching:")
for left_node, (right_node, weight) in matching.items():
    print(f"Left: {left_node}, Right: {right_node}, Weight: {weight}")
print("Greedy Objective:", alg)

# ocs
alg2s = []
for _ in range(10000): # 75% chance to get the optimal matching
    matching, alg2 = ocs_matching(left_nodes, right_nodes, edges)
    alg2s.append(alg2)
    for left_node, (right_node, weight) in matching.items():
        print(f"Left: {left_node}, Right: {right_node}, Weight: {weight}")
    print("OCS Objective:", alg2)

matching, obj = offline_matching(left_nodes, right_nodes, edges)
print("Optimal Matching:")
for left_node, (right_node, weight) in matching.items():
    print(f"Left: {left_node}, Right: {right_node}, Weight: {weight}")
print("Optimal Objective:", obj)

print("Competitive Ratio: Greedy ", alg / obj, "; OCS", np.mean(alg2s) / obj)

Greedy Matching:
Left: 1, Right: 1, Weight: 1
Greedy Objective: 1
Left: 2, Right: 1, Weight: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 2
Left: 1, Right: 2, Weight: 1
OCS Objective: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 1
Left: 2, Right: 1, Weight: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 2
Left: 1, Right: 2, Weight: 1
OCS Objective: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 1
Left: 2, Right: 1, Weight: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 2
Left: 1, Right: 2, Weight: 1
OCS Objective: 1
Left: 2, Right: 1, Weight: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 2
Left: 2, Right: 1, Weight: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 2
Left: 2, Right: 1, Weight: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 2
Left: 2, Right: 1, Weight: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 2
Left: 1, Right: 2, Weight: 1
OCS Objective: 1
Left: 2, Right: 1, Weight: 1
Left: 1, Right: 2, Weight: 1
OCS Objective: 2
Left: 1, Rig

### Erdös–Rényi upper triangular graphs

In [90]:
# Erdös–Rényi upper triangular graphs
verbose = False
n = 100
p = 2**(-1) # 2**(-4) 2**(-6)

def erdos_renyi(n, p):
    left_nodes = set([i for i in range(n)])
    right_nodes = set([i for i in range(n)])
    edges = {(i, i): 1 for i in range(n)}

    for i in range(n):
        for j in range(i+1, n): # +1
            if np.random.rand() < p:
                edges[(i, j)] = 1 # (unweighted)
    # print("edges", edges)
    # ## Add 0 weighted-edges
    for left_node in left_nodes:
        for right_node in right_nodes:
            if (left_node, right_node) not in edges:
                edges[(left_node, right_node)] = 0

    return left_nodes, right_nodes, edges

# left_nodes, right_nodes, edges = erdos_renyi(n, p)

random_cr_greedy = []
adversarial_cr_greedy = []
random_cr_ocs = []
adversarial_cr_ocs = []

for _ in range (10):
    np.random.seed(_)

    left_nodes, right_nodes, edges = erdos_renyi(n, p)

    ## Add 0 weighted-edges
    for left_node in left_nodes:
        for right_node in right_nodes:
            if (left_node, right_node) not in edges:
                edges[(left_node, right_node)] = 0

    competitive_ratios_greedy = []
    competitive_ratios_ocs = []
    # Try different orders of right nodes
    for _ in range(1):
        # right_nodes = np.random.permutation(list(right_nodes))
        right_nodes = sorted(list(right_nodes), reverse=True)
        # print("right_nodes", right_nodes)

        # Offline Matching
        _, obj = offline_matching(left_nodes, right_nodes, edges)

        # Greedy Algorithm
        greedy_obj = deterministic_greedy(left_nodes, right_nodes, edges)[1]
        competitive_ratios_greedy.append((greedy_obj / obj))
        # print("Greedy Objective:", greedy_obj)

        # OCS Algorithm
        ocs_obj = []
        for _ in range(5): # run 6 times to get the average competitive ratio
            ocs_obj.append(ocs_matching(left_nodes, right_nodes, edges)[1])
            # print("OCS Objective:", ocs_obj[-1])
        competitive_ratios_ocs.append(np.mean(ocs_obj) / obj)

    random_cr_greedy.append(np.mean(competitive_ratios_greedy))
    adversarial_cr_greedy.append(np.min(competitive_ratios_greedy))

    random_cr_ocs.append(np.mean(competitive_ratios_ocs))
    adversarial_cr_ocs.append(np.min(competitive_ratios_ocs))

    # print("Sample:", _)
    print("Random Greedy Competitive Ratio:", np.mean(competitive_ratios_greedy))
    print("Adversarial Greedy Competitive Ratio:", np.min(competitive_ratios_greedy))
    print("Random OCS Competitive Ratio:", np.mean(competitive_ratios_ocs))
    print("Adversarial OCS Competitive Ratio:", np.min(competitive_ratios_ocs))

print("Random Greedy Competitive Ratio:", np.mean(random_cr_greedy))
print("Adversarial Greedy Competitive Ratio:", np.mean(adversarial_cr_greedy))
print("Random OCS Competitive Ratio:", np.mean(random_cr_ocs))
print("Adversarial OCS Competitive Ratio:", np.mean(adversarial_cr_ocs))

Random Greedy Competitive Ratio: 0.5
Adversarial Greedy Competitive Ratio: 0.5
Random OCS Competitive Ratio: 0.522
Adversarial OCS Competitive Ratio: 0.522
Random Greedy Competitive Ratio: 0.51
Adversarial Greedy Competitive Ratio: 0.51
Random OCS Competitive Ratio: 0.518
Adversarial OCS Competitive Ratio: 0.518
Random Greedy Competitive Ratio: 0.51
Adversarial Greedy Competitive Ratio: 0.51
Random OCS Competitive Ratio: 0.526
Adversarial OCS Competitive Ratio: 0.526
Random Greedy Competitive Ratio: 0.51
Adversarial Greedy Competitive Ratio: 0.51
Random OCS Competitive Ratio: 0.524
Adversarial OCS Competitive Ratio: 0.524
Random Greedy Competitive Ratio: 0.5
Adversarial Greedy Competitive Ratio: 0.5
Random OCS Competitive Ratio: 0.524
Adversarial OCS Competitive Ratio: 0.524
Random Greedy Competitive Ratio: 0.5
Adversarial Greedy Competitive Ratio: 0.5
Random OCS Competitive Ratio: 0.524
Adversarial OCS Competitive Ratio: 0.524
Random Greedy Competitive Ratio: 0.52
Adversarial Greedy C

In [86]:
# simulate 10 times and calculate average competitive ratio
def evaluate(
        num_samples=100,
        num_nodes=100,
        num_edges=1000,
        max_weight=10,
        adv=1):

    cr_greedy = []
    cr_ocs = []

    for _ in range (num_samples):
        np.random.seed(_)

        # Generate random example (random orders)
        left_nodes = set([i for i in range(num_nodes)])
        right_nodes = set([i for i in range(num_nodes)])
        edges = {(np.random.choice(list(left_nodes)), np.random.choice(list(right_nodes))):
                  np.random.randint(1, max_weight) for _ in range(num_edges)}


        if adv == 1:
            # # adversarial case
            # # first half of the left nodes only connects to the right nodes having same index
            # # second half of the left nodes connect every first half of right nodes and right node having same index
            for left_node in left_nodes:
                for right_node in right_nodes:
                    if left_node >= right_node - 1:
                        edges[(left_node, right_node)] = right_node
                    else:
                        edges[(left_node, right_node)] = 0
        if adv == 2:
            # sort right nodes by the sum of edge weights, largest comes first
            right_nodes = [right_node for right_node, weight in
                           sorted([(right_node, sum([weight for edge, weight in edges.items() if edge[1] == right_node]))
                                   for right_node in right_nodes], key=lambda x: x[1])]
            # reassign the weights of edges based on the order of the right nodes, the i-th node's edges has weight i+1
            for i, right_node in enumerate(right_nodes):
                for left_node in left_nodes:
                    if (left_node, right_node) in edges:
                        edges[(left_node, right_node)] =  np.random.randint(1, i+2)
        if adv == 3:
            # reassign the weights of edges based on the order of the right nodes, the i-th node's edges has weight 1024+i
            for i, right_node in enumerate(right_nodes):
                for left_node in left_nodes:
                    if (left_node, right_node) in edges:
                        edges[(left_node, right_node)] = 1024+i

        # ## Add 0 weighted-edges
        for left_node in left_nodes:
            for right_node in right_nodes:
                if (left_node, right_node) not in edges:
                    edges[(left_node, right_node)] = 0
        # Offline Matching
        mat, obj = offline_matching(left_nodes, right_nodes, edges)
        # print("Optimal Matching:", mat)

        # Greedy Algorithm
        mat, greedy_obj = deterministic_greedy(left_nodes, right_nodes, edges)
        # print("Greedy Matching:", mat)

        # OCS Algorithm
        ocs_obj = np.mean([ocs_matching(left_nodes, right_nodes, edges)[1] for _ in range(5)])

        cr_greedy.append(greedy_obj / obj)
        cr_ocs.append(ocs_obj / obj)

        print("Sample: ", _, "Objectives:", obj, greedy_obj / obj, ocs_obj / obj)

    print(f"Random Greedy Competitive Ratio: min : {np.min(cr_greedy)} mean: {np.mean(cr_greedy)}")
    print(f"Random OCS Competitive Ratio: min : {np.min(cr_ocs)} mean: {np.mean(cr_ocs)}")


In [91]:
evaluate(num_samples=10, num_nodes=100, num_edges=50, max_weight=10)

Sample:  0 Objectives: 38761.0 1.0 0.9779159464410103
Sample:  1 Objectives: 36689.0 0.9718444220338521 0.9831066532203112
Sample:  2 Objectives: 38745.0 0.9183378500451671 0.9621989934185057
Sample:  3 Objectives: 39817.0 1.0 0.972890976216189
Sample:  4 Objectives: 39725.0 0.9740465701699181 1.0
Sample:  5 Objectives: 37450.0 1.0 0.9721869158878506
Sample:  6 Objectives: 35548.0 0.970068639585912 0.9760549116687297
Sample:  7 Objectives: 39632.0 1.0 1.0
Sample:  8 Objectives: 35627.0 1.0 0.9882785527830017
Sample:  9 Objectives: 34583.0 0.939768094150305 0.9938813868085475
Random Greedy Competitive Ratio: min : 0.9183378500451671 mean: 0.9774065575985154
Random OCS Competitive Ratio: min : 0.9621989934185057 mean: 0.9826514336444145


In [92]:
evaluate(num_samples=100, num_nodes=10, num_edges=50, max_weight=10)

Sample:  0 Objectives: 10285.0 1.0 0.8206319883325232
Sample:  1 Objectives: 10285.0 0.8003889158969373 0.7207972775887214
Sample:  2 Objectives: 10285.0 0.9004375303840545 0.8406028196402529
Sample:  3 Objectives: 10285.0 0.9004375303840545 0.9202333495381624
Sample:  4 Objectives: 10285.0 0.9004375303840545 0.8006806028196403
Sample:  5 Objectives: 10285.0 0.8001944579484687 0.7808070004861449
Sample:  6 Objectives: 10285.0 0.9003403014098201 0.8007389402041809
Sample:  7 Objectives: 10285.0 0.9001458434613515 0.7008264462809918
Sample:  8 Objectives: 10285.0 0.9004375303840545 0.8007778317938746
Sample:  9 Objectives: 10285.0 0.9001458434613515 0.720855614973262
Sample:  10 Objectives: 9258.0 1.0 0.8893713545042125
Sample:  11 Objectives: 10285.0 0.9004375303840545 0.800719494409334
Sample:  12 Objectives: 10285.0 0.7007292173067574 0.8804472532814779
Sample:  13 Objectives: 10285.0 1.0 0.8605736509479825
Sample:  14 Objectives: 10285.0 0.9004375303840545 0.8006417112299465
Sample: 

In [52]:
evaluate(num_samples=10000, num_nodes=10, num_edges=100, max_weight=10)

Sample:  1 Objectives: 50.0 0.9 0.92
Sample:  2 Objectives: 55.0 0.8181818181818182 0.9272727272727272
Sample:  6 Objectives: 46.0 0.8913043478260869 0.9347826086956522
Sample:  7 Objectives: 50.0 0.86 0.94
Sample:  9 Objectives: 48.0 0.8541666666666666 0.875
Sample:  15 Objectives: 51.0 0.9019607843137255 0.9803921568627451
Sample:  20 Objectives: 51.0 0.9019607843137255 0.9215686274509803
Sample:  31 Objectives: 46.0 0.9130434782608695 0.9347826086956522
Sample:  32 Objectives: 49.0 0.8571428571428571 0.9183673469387755
Sample:  40 Objectives: 50.0 0.84 0.86
Sample:  44 Objectives: 53.0 0.8113207547169812 0.8867924528301887
Sample:  51 Objectives: 48.0 0.9166666666666666 0.9375
Sample:  54 Objectives: 50.0 0.84 0.86
Sample:  67 Objectives: 54.0 0.8888888888888888 0.9444444444444444
Sample:  68 Objectives: 50.0 0.88 0.9
Sample:  69 Objectives: 48.0 0.8958333333333334 0.9375
Sample:  76 Objectives: 45.0 0.9555555555555556 0.9777777777777777
Sample:  80 Objectives: 52.0 0.92307692307692

In [53]:
verbose = False
evaluate(num_samples=1000, num_nodes=10, num_edges=10, max_weight=2)

Sample:  44 Objectives: 23.0 0.9565217391304348 1.0
Sample:  93 Objectives: 19.0 0.9473684210526315 1.0
Sample:  94 Objectives: 25.0 0.96 1.0
Sample:  133 Objectives: 29.0 0.9310344827586207 1.0
Sample:  207 Objectives: 23.0 0.8695652173913043 0.9130434782608695
Sample:  227 Objectives: 21.0 0.9047619047619048 0.9523809523809523
Sample:  326 Objectives: 30.0 0.8666666666666667 0.9666666666666667
Sample:  392 Objectives: 27.0 0.9259259259259259 1.0
Sample:  468 Objectives: 24.0 0.9583333333333334 1.0
Sample:  621 Objectives: 32.0 0.96875 1.0
Sample:  735 Objectives: 27.0 0.8888888888888888 1.0
Sample:  772 Objectives: 25.0 0.84 0.92
Sample:  806 Objectives: 33.0 0.9393939393939394 1.0
Sample:  894 Objectives: 27.0 0.9259259259259259 1.0
Random Greedy Competitive Ratio: min : 0.7037037037037037 mean: 0.9929407282378887
Random OCS Competitive Ratio: min : 0.6818181818181818 mean: 0.9794183041655851


In [57]:
evaluate(num_samples=1000, num_nodes=10, num_edges=100, max_weight=10)

Sample:  1 Objectives: 50.0 0.9 0.92
Sample:  2 Objectives: 55.0 0.8181818181818182 0.9272727272727272
Sample:  6 Objectives: 46.0 0.8913043478260869 0.9347826086956522
Sample:  7 Objectives: 50.0 0.86 0.94
Sample:  9 Objectives: 48.0 0.8541666666666666 0.875
Sample:  15 Objectives: 51.0 0.9019607843137255 0.9803921568627451
Sample:  20 Objectives: 51.0 0.9019607843137255 0.9215686274509803
Sample:  31 Objectives: 46.0 0.9130434782608695 0.9347826086956522
Sample:  32 Objectives: 49.0 0.8571428571428571 0.9183673469387755
Sample:  40 Objectives: 50.0 0.84 0.86
Sample:  44 Objectives: 53.0 0.8113207547169812 0.8867924528301887
Sample:  51 Objectives: 48.0 0.9166666666666666 0.9375
Sample:  54 Objectives: 50.0 0.84 0.86
Sample:  67 Objectives: 54.0 0.8888888888888888 0.9444444444444444
Sample:  68 Objectives: 50.0 0.88 0.9
Sample:  69 Objectives: 48.0 0.8958333333333334 0.9375
Sample:  76 Objectives: 45.0 0.9555555555555556 0.9777777777777777
Sample:  80 Objectives: 52.0 0.92307692307692

In [58]:
evaluate(num_samples=100, num_nodes=10, num_edges=100, max_weight=10)

Sample:  1 Objectives: 50.0 0.9 0.92
Sample:  2 Objectives: 55.0 0.8181818181818182 0.9272727272727272
Sample:  6 Objectives: 46.0 0.8913043478260869 0.9347826086956522
Sample:  7 Objectives: 50.0 0.86 0.94
Sample:  9 Objectives: 48.0 0.8541666666666666 0.875
Sample:  15 Objectives: 51.0 0.9019607843137255 0.9803921568627451
Sample:  20 Objectives: 51.0 0.9019607843137255 0.9215686274509803
Sample:  31 Objectives: 46.0 0.9130434782608695 0.9347826086956522
Sample:  32 Objectives: 49.0 0.8571428571428571 0.9183673469387755
Sample:  40 Objectives: 50.0 0.84 0.86
Sample:  44 Objectives: 53.0 0.8113207547169812 0.8867924528301887
Sample:  51 Objectives: 48.0 0.9166666666666666 0.9375
Sample:  54 Objectives: 50.0 0.84 0.86
Sample:  67 Objectives: 54.0 0.8888888888888888 0.9444444444444444
Sample:  68 Objectives: 50.0 0.88 0.9
Sample:  69 Objectives: 48.0 0.8958333333333334 0.9375
Sample:  76 Objectives: 45.0 0.9555555555555556 0.9777777777777777
Sample:  80 Objectives: 52.0 0.92307692307692

In [ ]:
# unweighted case
evaluate(num_samples=10, num_nodes=10, num_edges=10, max_weight=2)